### FE630 - Homework #2

**Author**: Sid Bhatia

**Date**: May 7th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Papa Momar Ndiaye

#### Topics

- Algebra & Optimization;
- Geometry of Efficient Frontiers;
- Applications of One-Fund & Two-Fund Theorems.

#### P1 - Optimization w/Equality Constraints (40 pts)